In [ ]:
AIRPORT = 'KMIA'
VFR_ALT_MAX = 2500

In [ ]:
import time, os, pathlib, csv, json, subprocess, sys, re
from datetime import datetime

import importlib.util as il
if None in [il.find_spec('FlightRadar24'), il.find_spec('numpy'), \
            il.find_spec('requests'), il.find_spec('pandas'), \
            il.find_spec('matplotlib'), il.find_spec('IPython'), \
            il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'FlightRadarAPI']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'numpy']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'requests']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pandas']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'IPython']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
os.system('cls')

from FlightRadar24 import FlightRadar24API
import numpy as np
import requests, urllib
import pandas as pd
import matplotlib as mpl, matplotlib.pyplot as plt
from IPython import display
import pyperclip

def pprint(dict):
    print(json.dumps(dict, indent=2))

def pcopy(dict):
    pyperclip.copy(json.dumps(dict, indent=2))

fr_api = FlightRadar24API()

ft_config = fr_api.get_flight_tracker_config()
# ft_config.gnd = '0'
ft_config.vehicles = '0'
fr_api.set_flight_tracker_config(ft_config)

In [ ]:
airport_data = fr_api.get_airport_details(AIRPORT)
airport_schedule = airport_data['airport']['pluginData']['schedule']
airport_pos = airport_data['airport']['pluginData']['details']['position']
arrivals = airport_schedule['arrivals']['data']
departures = airport_schedule['departures']['data']

In [ ]:
arrs, deps = [], []

for a in departures:
    a = a['flight']
    z = {}

    z['callsign'] = a['identification']['callsign']
    if z['callsign'] is None or a['aircraft'] is None:
        continue
    z['reg'] = a['aircraft']['registration']
    z['type'] = a['aircraft']['model']['code']

    z['origin'] = AIRPORT
    z['dest'] = a['airport']['destination']['code']['icao']
    
    z['gate'] = a['airport']['origin']['info']['gate']
    if z['gate'] is None:
        z['gate'] = 'NA'

    z['time'] = a['time']['scheduled']['departure']
    z['time-formatted'] = datetime.utcfromtimestamp(z['time']).strftime('%Y-%m-%d %H:%M:%S')

    deps.append(z)

for a in arrivals:
    a = a['flight']
    z = {}

    z['callsign'] = a['identification']['callsign']
    if z['callsign'] is None or a['aircraft'] is None:
        continue
    z['reg'] = a['aircraft']['registration']
    z['type'] = a['aircraft']['model']['code']

    z['origin'] = a['airport']['origin']['code']['icao']
    z['dest'] = AIRPORT

    z['gate'] = a['airport']['destination']['info']['gate']
    if z['gate'] is None:
        z['gate'] = 'NA'

    z['time'] = a['time']['real']['arrival'] 
    if a['time']['real']['arrival'] is None:
        z['time'] = a['time']['other']['eta']
    if z['time'] is None:
        continue
    
    z['time-formatted'] = datetime.utcfromtimestamp(z['time']).strftime('%Y-%m-%d %H:%M:%S')

    arrs.append(z)

deps = sorted_flights = sorted(deps, key=lambda x: x['time'])
arrs = sorted_flights = sorted(arrs, key=lambda x: x['time'])

pcopy(deps)
pcopy(arrs)

In [ ]:
raise Exception('Experimental non-scheduled data')
def get_flight_details(id):
    base_url = 'https://data-live.flightradar24.com/clickhandler/?flight={}'
    data_url = base_url.format(id)
    req = urllib.request.Request(data_url, headers={'User-Agent': 'Magic Browser'})
    
    n_retry = 0
    con = None
    while con is None:
        try:
            con = urllib.request.urlopen(req, timeout=(50 + 10 * n_retry))
        except urllib.request.HTTPError as e:
            if n_retry > 5:
                return None
            n_retry += 1
    return json.loads(con.read().decode('utf8'))

bounds = fr_api.get_bounds_by_point(airport_pos['latitude'], airport_pos['longitude'], 9260)
bound_flights = fr_api.get_flights(bounds = bounds)

flights = arrs + deps
arrdep_callsigns = [flight['callsign'] for flight in flights]
f = []

for flight in bound_flights:
    if flight.altitude > VFR_ALT_MAX or flight.callsign is None:
        continue
    a = get_flight_details(flight.id)
    z = {}
    
    z['callsign'] = a['identification']['callsign']
        
    if z['callsign'] is None or a['aircraft'] is None \
        or z['callsign'] == 'Blocked' or z['callsign'] in arrdep_callsigns:
        continue
    
    z['reg'] = a['aircraft']['registration']
    z['type'] = a['aircraft']['model']['code']
    print(z)
    print(flight)
    f.append(z)

pcopy(f)